In [2]:
import pandas as pd

# Step 1: Load your data using the provided file paths
matchdays_df = pd.read_csv('/Users/andreas/Desktop/02 - Moneyball/01 - Streamlit/Moneyball/data/merged_matchdays_copy.csv')
rankings_df = pd.read_csv('/Users/andreas/Desktop/02 - Moneyball/01 - Streamlit/Moneyball/data/merged_rankings_copy.csv')

In [3]:
matchdays_df.tail()

,Matchday,Date,Time,Location,Home Team,Away Team,Home Goals,Away Goals,Result,Geri Home Goals Predicted,...,Gerd Result Predicted,Samson Home Goals Predicted,Samson Away Goals Predicted,Samson Result Predicted,Moddy Home Goals Predicted,Moddy Away Goals Predicted,Moddy Result Predicted,Hermann Home Goals Predicted,Hermann Away Goals Predicted,Hermann Result Predicted
301,34,18/05/2024,13:30,Volkswagen Arena,WOB,M05,1,3,Away Win,2,...,Home Win,1,0,Home Win,2,1,Home Win,1,2,Away Win
302,34,18/05/2024,13:30,Deutsche Bank Park,SGE,RBL,2,2,Tie,1,...,Away Win,0,1,Away Win,2,2,Tie,1,2,Away Win
303,34,18/05/2024,13:30,BayArena,B04,FCA,2,1,Home Win,3,...,Home Win,2,0,Home Win,4,1,Home Win,4,1,Home Win
304,34,18/05/2024,13:30,Stadion an der Alten Försterei,FCU,SCF,2,1,Home Win,1,...,Tie,1,0,Home Win,1,2,Away Win,2,2,Tie
305,34,18/05/2024,13:30,SIGNAL IDUNA PARK,BVB,D98,4,0,Home Win,2,...,Home Win,2,0,Home Win,5,1,Home Win,4,0,Home Win


In [4]:
rankings_df.tail()

,Spieltag,Name,Punkte,Spieltagsplatzierung,Spieltagssieger,Gesamtspieltagssiege,Gesamtpunkte,Rang,Matchday
233,34,Johnny,10,5,0.0,4.33,390,3,34
234,34,Gerd,10,5,0.0,6.33,369,4,34
235,34,Samson,13,3,0.0,6.25,363,5,34
236,34,Moddy,11,4,0.0,3.33,349,6,34
237,34,Hermann,14,2,0.0,2.75,338,7,34


In [11]:
# Define the logic to calculate Wotcins' predictions
def calculate_wotcins_for_match(match_data, rankings):
    # Calculate the tendencies (home, away, tie) for all players
    tendencies = {}
    
    # Iterate through each player's prediction
    for index in match_data.index:
        if 'Home Goals Predicted' in index and 'Result Predicted' not in index:
            player_name = index.split(' ')[0]
            home_goals_pred = match_data[f'{player_name} Home Goals Predicted']
            away_goals_pred = match_data[f'{player_name} Away Goals Predicted']
            if home_goals_pred > away_goals_pred:
                tendencies[player_name] = 'Home Win'
            elif home_goals_pred < away_goals_pred:
                tendencies[player_name] = 'Away Win'
            else:
                tendencies[player_name] = 'Tie'
    
    # Count the tendencies
    tendency_counts = pd.Series(tendencies.values()).value_counts()

    # Determine the majority tendency
    if len(tendency_counts) > 1 and tendency_counts.iloc[0] == tendency_counts.iloc[1]:
        # Tie-breaker using rankings
        tied_tendencies = tendency_counts[tendency_counts == tendency_counts.iloc[0]].index.tolist()
        for player in rankings['Name']:
            player_tendency = tendencies.get(player)
            if player_tendency in tied_tendencies:
                majority_tendency = player_tendency
                break
    else:
        majority_tendency = tendency_counts.idxmax()

    # Filter the players who predicted the majority tendency
    relevant_players = [player for player, tendency in tendencies.items() if tendency == majority_tendency]
    relevant_predictions = match_data[[f'{player} Home Goals Predicted' for player in relevant_players]]

    # Calculate Wotcins' average predictions
    avg_home_goals = relevant_predictions.mean()
    avg_away_goals = match_data[[f'{player} Away Goals Predicted' for player in relevant_players]].mean()

    # Round the predictions to the nearest integer
    wotcins_prediction = {
        'Home Goals Predicted': round(avg_home_goals),
        'Away Goals Predicted': round(avg_away_goals),
        'Result Predicted': majority_tendency
    }

    return wotcins_prediction

# Apply the logic to each match
wotcins_predictions = []

for i, row in matchdays_df.iterrows():
    match_data = row.filter(like='Predicted')
    match_name = f"{row['Home Team']} - {row['Away Team']}"
    wotcins_prediction = calculate_wotcins_for_match(match_data, rankings_df)
    wotcins_predictions.append(wotcins_prediction)

# Convert the list of dictionaries into a DataFrame
wotcins_df = pd.DataFrame(wotcins_predictions)

# Add Wotcins' predictions to the original DataFrame
matchdays_df['Wotcins Home Goals Predicted'] = wotcins_df['Home Goals Predicted']
matchdays_df['Wotcins Away Goals Predicted'] = wotcins_df['Away Goals Predicted']
matchdays_df['Wotcins Result Predicted'] = wotcins_df['Result Predicted']

# Preview the updated data
matchdays_df.head()

,Matchday,Date,Time,Location,Home Team,Away Team,Home Goals,Away Goals,Result,Geri Home Goals Predicted,...,Samson Result Predicted,Moddy Home Goals Predicted,Moddy Away Goals Predicted,Moddy Result Predicted,Hermann Home Goals Predicted,Hermann Away Goals Predicted,Hermann Result Predicted,Wotcins Home Goals Predicted,Wotcins Away Goals Predicted,Wotcins Result Predicted
0,1,18/08/2023,18:30,wohninvest WESERSTADION,SVW,FCB,0,4,Away Win,1,...,Away Win,1,3,Away Win,1,3,Away Win,1,3,Away Win
1,1,19/08/2023,13:30,Mercedes-Benz Arena,VFB,BOC,5,0,Home Win,0,...,Away Win,1,2,Away Win,2,0,Home Win,2,0,Home Win
2,1,19/08/2023,13:30,WWK ARENA,FCA,BMG,4,4,Tie,0,...,Tie,2,2,Tie,0,1,Away Win,1,2,Away Win
3,1,19/08/2023,13:30,PreZero Arena,TSG,SCF,1,2,Away Win,0,...,Away Win,1,2,Away Win,1,2,Away Win,1,2,Away Win
4,1,19/08/2023,13:30,Volkswagen Arena,WOB,HDH,2,0,Home Win,2,...,Tie,3,1,Home Win,1,1,Tie,2,0,Home Win


In [29]:
matchdays_df.head(27)

,Matchday,Date,Time,Location,Home Team,Away Team,Home Goals,Away Goals,Result,Geri Home Goals Predicted,...,Samson Result Predicted,Moddy Home Goals Predicted,Moddy Away Goals Predicted,Moddy Result Predicted,Hermann Home Goals Predicted,Hermann Away Goals Predicted,Hermann Result Predicted,Wotcins Home Goals Predicted,Wotcins Away Goals Predicted,Wotcins Result Predicted
0,1,18/08/2023,18:30,wohninvest WESERSTADION,SVW,FCB,0,4,Away Win,1,...,Away Win,1,3,Away Win,1,3,Away Win,1,3,Away Win
1,1,19/08/2023,13:30,Mercedes-Benz Arena,VFB,BOC,5,0,Home Win,0,...,Away Win,1,2,Away Win,2,0,Home Win,2,0,Home Win
2,1,19/08/2023,13:30,WWK ARENA,FCA,BMG,4,4,Tie,0,...,Tie,2,2,Tie,0,1,Away Win,1,2,Away Win
3,1,19/08/2023,13:30,PreZero Arena,TSG,SCF,1,2,Away Win,0,...,Away Win,1,2,Away Win,1,2,Away Win,1,2,Away Win
4,1,19/08/2023,13:30,Volkswagen Arena,WOB,HDH,2,0,Home Win,2,...,Tie,3,1,Home Win,1,1,Tie,2,0,Home Win
5,1,19/08/2023,13:30,BayArena,B04,RBL,3,2,Home Win,1,...,Away Win,2,2,Tie,2,2,Tie,2,2,Tie
6,1,19/08/2023,16:30,SIGNAL IDUNA PARK,BVB,KOE,1,0,Home Win,2,...,Home Win,1,2,Away Win,3,0,Home Win,3,1,Home Win
7,1,20/08/2023,13:30,Stadion an der Alten Försterei,FCU,M05,4,1,Home Win,2,...,Home Win,2,1,Home Win,2,0,Home Win,2,0,Home Win
8,1,20/08/2023,15:30,Deutsche Bank Park,SGE,D98,1,0,Home Win,2,...,Home Win,3,0,Home Win,3,1,Home Win,3,0,Home Win
9,2,25/08/2023,18:30,Red Bull Arena,RBL,VFB,5,1,Home Win,2,...,Home Win,3,1,Home Win,3,0,Home Win,3,1,Home Win


In [12]:
matchdays_df.tail()

,Matchday,Date,Time,Location,Home Team,Away Team,Home Goals,Away Goals,Result,Geri Home Goals Predicted,...,Samson Result Predicted,Moddy Home Goals Predicted,Moddy Away Goals Predicted,Moddy Result Predicted,Hermann Home Goals Predicted,Hermann Away Goals Predicted,Hermann Result Predicted,Wotcins Home Goals Predicted,Wotcins Away Goals Predicted,Wotcins Result Predicted
301,34,18/05/2024,13:30,Volkswagen Arena,WOB,M05,1,3,Away Win,2,...,Home Win,2,1,Home Win,1,2,Away Win,2,1,Home Win
302,34,18/05/2024,13:30,Deutsche Bank Park,SGE,RBL,2,2,Tie,1,...,Away Win,2,2,Tie,1,2,Away Win,1,2,Away Win
303,34,18/05/2024,13:30,BayArena,B04,FCA,2,1,Home Win,3,...,Home Win,4,1,Home Win,4,1,Home Win,3,1,Home Win
304,34,18/05/2024,13:30,Stadion an der Alten Försterei,FCU,SCF,2,1,Home Win,1,...,Home Win,1,2,Away Win,2,2,Tie,2,2,Tie
305,34,18/05/2024,13:30,SIGNAL IDUNA PARK,BVB,D98,4,0,Home Win,2,...,Home Win,5,1,Home Win,4,0,Home Win,3,0,Home Win


In [20]:
# Function to calculate points based on the rules provided
def calculate_points(pred_home, pred_away, actual_home, actual_away):
    if pred_home == actual_home and pred_away == actual_away:
        return 4  # Exact score
    elif (pred_home - pred_away) == (actual_home - actual_away):
        return 3  # Correct goal difference
    elif (pred_home > pred_away and actual_home > actual_away) or (pred_home < pred_away and actual_away > actual_home) or (pred_home == pred_away and actual_home == actual_away):
        return 2  # Correct outcome/tendency
    else:
        return 0  # Wrong prediction

# Calculate Wotcins' points for each matchday
wotcins_points = []
for matchday in matchdays_df['Matchday'].unique():
    matchday_data = matchdays_df[matchdays_df['Matchday'] == matchday]
    wotcins_matchday_points = 0
    for _, row in matchday_data.iterrows():
        points = calculate_points(row['Wotcins Home Goals Predicted'], row['Wotcins Away Goals Predicted'],
                                  row['Home Goals'], row['Away Goals'])
        wotcins_matchday_points += points
    wotcins_points.append({'Spieltag': matchday, 'Name': 'Wotcins', 'Punkte': wotcins_matchday_points})

# Convert Wotcins' points to a DataFrame
wotcins_df = pd.DataFrame(wotcins_points)

# Append Wotcins to the existing rankings dataframe using pd.concat
rankings_df = pd.concat([rankings_df, wotcins_df], ignore_index=True)

# Recalculate the necessary columns
updated_rankings = []
for matchday in rankings_df['Spieltag'].unique():
    matchday_data = rankings_df[rankings_df['Spieltag'] == matchday].copy()
    
    # Sort players by points and recalculate Spieltagsplatzierung (matchday ranking)
    matchday_data = matchday_data.sort_values(by='Punkte', ascending=False)
    matchday_data['Spieltagsplatzierung'] = range(1, len(matchday_data) + 1)
    
    # Recalculate Gesamtpunkte (cumulative points)
    if matchday == 1:
        matchday_data['Gesamtpunkte'] = matchday_data['Punkte']
    else:
        prev_totals = rankings_df[rankings_df['Spieltag'] < matchday].groupby('Name')['Gesamtpunkte'].sum().reset_index()
        matchday_data = matchday_data.merge(prev_totals, on='Name', how='left', suffixes=('', '_prev'))
        matchday_data['Gesamtpunkte'] = matchday_data['Gesamtpunkte'].fillna(0) + matchday_data['Punkte']
    
    # Rank players by Gesamtpunkte
    matchday_data = matchday_data.sort_values(by='Gesamtpunkte', ascending=False)
    matchday_data['Rang'] = range(1, len(matchday_data) + 1)
    
    updated_rankings.append(matchday_data)

# Concatenate all matchday rankings
updated_rankings_df = pd.concat(updated_rankings, ignore_index=True)

# Preview the updated data
updated_rankings_df.head()

,Spieltag,Name,Punkte,Spieltagsplatzierung,Spieltagssieger,Gesamtspieltagssiege,Gesamtpunkte,Rang,Matchday,Gesamtpunkte_prev
0,1,Wotcins,18,1,NaN,NaN,18.0,1,NaN,NaN
1,1,Wotcins,18,2,NaN,NaN,18.0,2,NaN,NaN
2,1,Samson,16,3,1.0,1.0,16.0,3,1.0,NaN
3,1,Geri,15,4,0.0,0.0,15.0,4,1.0,NaN
4,1,Moddy,15,5,0.0,0.0,15.0,5,1.0,NaN


In [22]:
# Function to calculate points based on the rules provided
def calculate_points(pred_home, pred_away, actual_home, actual_away):
    if pred_home == actual_home and pred_away == actual_away:
        return 4  # Exact score
    elif (pred_home - pred_away) == (actual_home - actual_away):
        return 3  # Correct goal difference
    elif (pred_home > pred_away and actual_home > actual_away) or (pred_home < pred_away and actual_home < actual_away) or (pred_home == pred_away and actual_home == actual_away):
        return 2  # Correct outcome/tendency
    else:
        return 0  # Wrong prediction

# Calculate Wotcins' points for each matchday
wotcins_points = []
for matchday in matchdays_df['Matchday'].unique():
    matchday_data = matchdays_df[matchdays_df['Matchday'] == matchday]
    wotcins_matchday_points = 0
    for _, row in matchday_data.iterrows():
        points = calculate_points(row['Wotcins Home Goals Predicted'], row['Wotcins Away Goals Predicted'],
                                  row['Home Goals'], row['Away Goals'])
        wotcins_matchday_points += points
    wotcins_points.append({'Spieltag': matchday, 'Name': 'Wotcins', 'Punkte': wotcins_matchday_points})

# Convert Wotcins' points to a DataFrame
wotcins_df = pd.DataFrame(wotcins_points)

# Append Wotcins to the existing rankings dataframe using pd.concat
rankings_df = pd.concat([rankings_df, wotcins_df], ignore_index=True)

# Recalculate the necessary columns
updated_rankings = []
for matchday in rankings_df['Spieltag'].unique():
    matchday_data = rankings_df[rankings_df['Spieltag'] == matchday].copy()
    
    # Drop duplicate entries if they exist (e.g., multiple Wotcins entries)
    matchday_data = matchday_data.drop_duplicates(subset=['Name'], keep='first')
    
    # Sort players by points and recalculate Spieltagsplatzierung (matchday ranking)
    matchday_data = matchday_data.sort_values(by='Punkte', ascending=False)
    matchday_data['Spieltagsplatzierung'] = range(1, len(matchday_data) + 1)
    
    # Recalculate Gesamtpunkte (cumulative points)
    if matchday == 1:
        matchday_data['Gesamtpunkte'] = matchday_data['Punkte']
    else:
        prev_totals = rankings_df[rankings_df['Spieltag'] < matchday].groupby('Name')['Gesamtpunkte'].sum().reset_index()
        matchday_data = matchday_data.merge(prev_totals, on='Name', how='left', suffixes=('', '_prev'))
        matchday_data['Gesamtpunkte'] = matchday_data['Gesamtpunkte'].fillna(0) + matchday_data['Punkte']
    
    # Rank players by Gesamtpunkte
    matchday_data = matchday_data.sort_values(by='Gesamtpunkte', ascending=False)
    matchday_data['Rang'] = range(1, len(matchday_data) + 1)
    
    # Select only the relevant columns
    matchday_data = matchday_data[['Spieltag', 'Name', 'Punkte', 'Spieltagsplatzierung', 'Gesamtpunkte', 'Rang']]
    
    updated_rankings.append(matchday_data)

# Concatenate all matchday rankings
updated_rankings_df = pd.concat(updated_rankings, ignore_index=True)

# Preview the updated data
updated_rankings_df.head()


,Spieltag,Name,Punkte,Spieltagsplatzierung,Gesamtpunkte,Rang
0,1,Wotcins,18,1,18.0,1
1,1,Samson,16,2,16.0,2
2,1,Geri,15,3,15.0,3
3,1,Moddy,15,4,15.0,4
4,1,Hermann,14,5,14.0,5


In [25]:
updated_rankings_df.tail(16)

,Spieltag,Name,Punkte,Spieltagsplatzierung,Gesamtpunkte,Rang
256,33,Johnny,13,2,393.0,1
257,33,Geri,9,6,391.0,2
258,33,Andreas,11,5,385.0,3
259,33,Gerd,18,1,377.0,4
260,33,Samson,8,8,358.0,5
261,33,Moddy,9,7,347.0,6
262,33,Hermann,13,3,337.0,7
263,33,Wotcins,13,4,13.0,8
264,34,Andreas,19,1,412.0,1
265,34,Geri,10,5,402.0,2


In [26]:
# Function to calculate points based on the rules provided
def calculate_points(pred_home, pred_away, actual_home, actual_away):
    if pred_home == actual_home and pred_away == actual_away:
        return 4  # Exact score
    elif (pred_home - pred_away) == (actual_home - actual_away):
        return 3  # Correct goal difference
    elif (pred_home > pred_away and actual_home > actual_away) or (pred_home < pred_away and actual_home < actual_away) or (pred_home == pred_away and actual_home == actual_away):
        return 2  # Correct outcome/tendency
    else:
        return 0  # Wrong prediction

# Calculate Wotcins' points for each matchday
wotcins_points = []
for matchday in matchdays_df['Matchday'].unique():
    matchday_data = matchdays_df[matchdays_df['Matchday'] == matchday]
    wotcins_matchday_points = 0
    for _, row in matchday_data.iterrows():
        points = calculate_points(row['Wotcins Home Goals Predicted'], row['Wotcins Away Goals Predicted'],
                                  row['Home Goals'], row['Away Goals'])
        wotcins_matchday_points += points
    wotcins_points.append({'Spieltag': matchday, 'Name': 'Wotcins', 'Punkte': wotcins_matchday_points})

# Convert Wotcins' points to a DataFrame
wotcins_df = pd.DataFrame(wotcins_points)

# Append Wotcins to the existing rankings dataframe using pd.concat
rankings_df = pd.concat([rankings_df, wotcins_df], ignore_index=True)

# Initialize a cumulative points dictionary
cumulative_points = {player: 0 for player in rankings_df['Name'].unique()}

# Recalculate the necessary columns
updated_rankings = []
for matchday in rankings_df['Spieltag'].unique():
    matchday_data = rankings_df[rankings_df['Spieltag'] == matchday].copy()
    
    # Drop duplicate entries if they exist (e.g., multiple Wotcins entries)
    matchday_data = matchday_data.drop_duplicates(subset=['Name'], keep='first')
    
    # Sort players by points and recalculate Spieltagsplatzierung (matchday ranking)
    matchday_data = matchday_data.sort_values(by='Punkte', ascending=False)
    matchday_data['Spieltagsplatzierung'] = range(1, len(matchday_data) + 1)
    
    # Update cumulative points and assign them to Gesamtpunkte
    for idx, row in matchday_data.iterrows():
        player = row['Name']
        cumulative_points[player] += row['Punkte']
        matchday_data.at[idx, 'Gesamtpunkte'] = cumulative_points[player]
    
    # Rank players by cumulative points (Gesamtpunkte)
    matchday_data = matchday_data.sort_values(by='Gesamtpunkte', ascending=False)
    matchday_data['Rang'] = range(1, len(matchday_data) + 1)
    
    # Select only the relevant columns
    matchday_data = matchday_data[['Spieltag', 'Name', 'Punkte', 'Spieltagsplatzierung', 'Gesamtpunkte', 'Rang']]
    
    updated_rankings.append(matchday_data)

# Concatenate all matchday rankings
updated_rankings_df = pd.concat(updated_rankings, ignore_index=True)

# Preview the updated data
updated_rankings_df.head()


,Spieltag,Name,Punkte,Spieltagsplatzierung,Gesamtpunkte,Rang
0,1,Wotcins,18,1,18.0,1
1,1,Samson,16,2,16.0,2
2,1,Geri,15,3,15.0,3
3,1,Moddy,15,4,15.0,4
4,1,Hermann,14,5,14.0,5


In [28]:
# Preview the updated data
updated_rankings_df.tail(16)


,Spieltag,Name,Punkte,Spieltagsplatzierung,Gesamtpunkte,Rang
256,33,Geri,9,6,382.0,1
257,33,Johnny,13,2,380.0,2
258,33,Andreas,11,5,374.0,3
259,33,Gerd,18,1,359.0,4
260,33,Wotcins,13,4,358.0,5
261,33,Samson,8,8,350.0,6
262,33,Moddy,9,7,338.0,7
263,33,Hermann,13,3,324.0,8
264,34,Andreas,19,1,393.0,1
265,34,Geri,10,5,392.0,2


In [33]:
import pandas as pd
import numpy as np

# Load the matchdays and rankings dataframes
matchdays_df = pd.read_csv('/Users/andreas/Desktop/02 - Moneyball/01 - Streamlit/Moneyball/data/merged_matchdays_copy.csv')
rankings_df = pd.read_csv('/Users/andreas/Desktop/02 - Moneyball/01 - Streamlit/Moneyball/data/merged_rankings_copy.csv')

# Calculate Wotcins' predictions based on the average of all players' predictions
wotcins_predictions = []
for matchday in matchdays_df['Matchday'].unique():
    matchday_data = matchdays_df[matchdays_df['Matchday'] == matchday]
    
    for _, row in matchday_data.iterrows():
        # Get all predicted Home and Away goals for this match
        player_home_goals = row.filter(like='Home Goals Predicted')
        player_away_goals = row.filter(like='Away Goals Predicted')
        
        # Calculate the average predicted goals
        avg_home_goals = np.round(player_home_goals.mean())
        avg_away_goals = np.round(player_away_goals.mean())
        
        # Determine the result based on Wotcins' prediction
        if avg_home_goals > avg_away_goals:
            wotcins_result = 'Home Win'
        elif avg_home_goals < avg_away_goals:
            wotcins_result = 'Away Win'
        else:
            wotcins_result = 'Tie'
        
        wotcins_predictions.append({
            'Matchday': matchday,
            'Home Team': row['Home Team'],
            'Away Team': row['Away Team'],
            'Wotcins Home Goals Predicted': int(avg_home_goals),
            'Wotcins Away Goals Predicted': int(avg_away_goals),
            'Wotcins Result Predicted': wotcins_result
        })

# Convert the list of Wotcins' predictions into a DataFrame
wotcins_df = pd.DataFrame(wotcins_predictions)

# Merge Wotcins' predictions into the original matchdays dataframe
matchdays_df = matchdays_df.merge(wotcins_df, on=['Matchday', 'Home Team', 'Away Team'], how='left')

# Calculate Wotcins' points for each matchday
wotcins_points = []
for matchday in matchdays_df['Matchday'].unique():
    matchday_data = matchdays_df[matchdays_df['Matchday'] == matchday]
    wotcins_matchday_points = 0
    for _, row in matchday_data.iterrows():
        points = calculate_points(row['Wotcins Home Goals Predicted'], row['Wotcins Away Goals Predicted'],
                                  row['Home Goals'], row['Away Goals'])
        wotcins_matchday_points += points
    wotcins_points.append({'Spieltag': matchday, 'Name': 'Wotcins', 'Punkte': wotcins_matchday_points})

# Convert Wotcins' points to a DataFrame
wotcins_points_df = pd.DataFrame(wotcins_points)

# Append Wotcins to the existing rankings dataframe using pd.concat
rankings_df = pd.concat([rankings_df, wotcins_points_df], ignore_index=True)

# Initialize a cumulative points dictionary
cumulative_points = {player: 0 for player in rankings_df['Name'].unique()}

# Recalculate the necessary columns
updated_rankings = []
for matchday in rankings_df['Spieltag'].unique():
    matchday_data = rankings_df[rankings_df['Spieltag'] == matchday].copy()
    
    # Drop duplicate entries if they exist (e.g., multiple Wotcins entries)
    matchday_data = matchday_data.drop_duplicates(subset=['Name'], keep='first')
    
    # Sort players by points and recalculate Spieltagsplatzierung (matchday ranking)
    matchday_data = matchday_data.sort_values(by='Punkte', ascending=False)
    matchday_data['Spieltagsplatzierung'] = range(1, len(matchday_data) + 1)
    
    # Update cumulative points and assign them to Gesamtpunkte
    for idx, row in matchday_data.iterrows():
        player = row['Name']
        cumulative_points[player] += row['Punkte']
        matchday_data.at[idx, 'Gesamtpunkte'] = cumulative_points[player]
    
    # Rank players by cumulative points (Gesamtpunkte)
    matchday_data = matchday_data.sort_values(by='Gesamtpunkte', ascending=False)
    matchday_data['Rang'] = range(1, len(matchday_data) + 1)
    
    # Select only the relevant columns
    matchday_data = matchday_data[['Spieltag', 'Name', 'Punkte', 'Spieltagsplatzierung', 'Gesamtpunkte', 'Rang']]
    
    updated_rankings.append(matchday_data)

# Concatenate all matchday rankings
updated_rankings_df = pd.concat(updated_rankings, ignore_index=True)

# Preview the updated data
updated_rankings_df.tail(8)


,Spieltag,Name,Punkte,Spieltagsplatzierung,Gesamtpunkte,Rang
264,34,Andreas,19,1,393.0,1
265,34,Geri,10,5,392.0,2
266,34,Wotcins,10,8,392.0,3
267,34,Johnny,10,6,390.0,4
268,34,Gerd,10,7,369.0,5
269,34,Samson,13,3,363.0,6
270,34,Moddy,11,4,349.0,7
271,34,Hermann,14,2,338.0,8


In [37]:
updated_rankings_df.tail(16)

,Spieltag,Name,Punkte,Spieltagsplatzierung,Gesamtpunkte,Rang
256,33,Wotcins,14,2,382.0,1
257,33,Geri,9,6,382.0,2
258,33,Johnny,13,3,380.0,3
259,33,Andreas,11,5,374.0,4
260,33,Gerd,18,1,359.0,5
261,33,Samson,8,8,350.0,6
262,33,Moddy,9,7,338.0,7
263,33,Hermann,13,4,324.0,8
264,34,Andreas,19,1,393.0,1
265,34,Geri,10,5,392.0,2


In [34]:
import pandas as pd
import numpy as np

# Load the matchdays and rankings dataframes
matchdays_df = pd.read_csv('/Users/andreas/Desktop/02 - Moneyball/01 - Streamlit/Moneyball/data/merged_matchdays_copy.csv')
rankings_df = pd.read_csv('/Users/andreas/Desktop/02 - Moneyball/01 - Streamlit/Moneyball/data/merged_rankings_copy.csv')

# Calculate Wotcins' predictions based on the average of all players' predictions
wotcins_predictions = []
for matchday in matchdays_df['Matchday'].unique():
    matchday_data = matchdays_df[matchdays_df['Matchday'] == matchday]
    
    for _, row in matchday_data.iterrows():
        # Get all predicted Home and Away goals for this match
        player_home_goals = row.filter(like='Home Goals Predicted')
        player_away_goals = row.filter(like='Away Goals Predicted')
        
        # Calculate the average predicted goals
        avg_home_goals = np.round(player_home_goals.mean())
        avg_away_goals = np.round(player_away_goals.mean())
        
        # Determine the result based on Wotcins' prediction
        if avg_home_goals > avg_away_goals:
            wotcins_result = 'Home Win'
        elif avg_home_goals < avg_away_goals:
            wotcins_result = 'Away Win'
        else:
            wotcins_result = 'Tie'
        
        wotcins_predictions.append({
            'Matchday': matchday,
            'Home Team': row['Home Team'],
            'Away Team': row['Away Team'],
            'Wotcins Home Goals Predicted': int(avg_home_goals),
            'Wotcins Away Goals Predicted': int(avg_away_goals),
            'Wotcins Result Predicted': wotcins_result
        })

# Convert the list of Wotcins' predictions into a DataFrame
wotcins_df = pd.DataFrame(wotcins_predictions)

# Merge Wotcins' predictions into the original matchdays dataframe
updated_matchdays_df = matchdays_df.merge(wotcins_df, on=['Matchday', 'Home Team', 'Away Team'], how='left')

# Preview the updated matchdays data with Wotcins' predictions
updated_matchdays_df.head()

,Matchday,Date,Time,Location,Home Team,Away Team,Home Goals,Away Goals,Result,Geri Home Goals Predicted,...,Samson Result Predicted,Moddy Home Goals Predicted,Moddy Away Goals Predicted,Moddy Result Predicted,Hermann Home Goals Predicted,Hermann Away Goals Predicted,Hermann Result Predicted,Wotcins Home Goals Predicted,Wotcins Away Goals Predicted,Wotcins Result Predicted
0,1,18/08/2023,18:30,wohninvest WESERSTADION,SVW,FCB,0,4,Away Win,1,...,Away Win,1,3,Away Win,1,3,Away Win,1,3,Away Win
1,1,19/08/2023,13:30,Mercedes-Benz Arena,VFB,BOC,5,0,Home Win,0,...,Away Win,1,2,Away Win,2,0,Home Win,1,1,Tie
2,1,19/08/2023,13:30,WWK ARENA,FCA,BMG,4,4,Tie,0,...,Tie,2,2,Tie,0,1,Away Win,1,2,Away Win
3,1,19/08/2023,13:30,PreZero Arena,TSG,SCF,1,2,Away Win,0,...,Away Win,1,2,Away Win,1,2,Away Win,1,2,Away Win
4,1,19/08/2023,13:30,Volkswagen Arena,WOB,HDH,2,0,Home Win,2,...,Tie,3,1,Home Win,1,1,Tie,2,0,Home Win


In [35]:
updated_matchdays_df.tail(9)

,Matchday,Date,Time,Location,Home Team,Away Team,Home Goals,Away Goals,Result,Geri Home Goals Predicted,...,Samson Result Predicted,Moddy Home Goals Predicted,Moddy Away Goals Predicted,Moddy Result Predicted,Hermann Home Goals Predicted,Hermann Away Goals Predicted,Hermann Result Predicted,Wotcins Home Goals Predicted,Wotcins Away Goals Predicted,Wotcins Result Predicted
297,34,18/05/2024,13:30,Voith-Arena,HDH,KOE,4,1,Home Win,1,...,Home Win,1,3,Away Win,2,0,Home Win,2,1,Home Win
298,34,18/05/2024,13:30,Mercedes-Benz Arena,VFB,BMG,4,0,Home Win,2,...,Home Win,1,2,Away Win,3,1,Home Win,2,1,Home Win
299,34,18/05/2024,13:30,wohninvest WESERSTADION,SVW,BOC,4,1,Home Win,2,...,Home Win,2,0,Home Win,2,1,Home Win,2,1,Home Win
300,34,18/05/2024,13:30,PreZero Arena,TSG,FCB,4,2,Home Win,1,...,Tie,1,3,Away Win,2,2,Tie,1,2,Away Win
301,34,18/05/2024,13:30,Volkswagen Arena,WOB,M05,1,3,Away Win,2,...,Home Win,2,1,Home Win,1,2,Away Win,2,1,Home Win
302,34,18/05/2024,13:30,Deutsche Bank Park,SGE,RBL,2,2,Tie,1,...,Away Win,2,2,Tie,1,2,Away Win,1,2,Away Win
303,34,18/05/2024,13:30,BayArena,B04,FCA,2,1,Home Win,3,...,Home Win,4,1,Home Win,4,1,Home Win,3,1,Home Win
304,34,18/05/2024,13:30,Stadion an der Alten Försterei,FCU,SCF,2,1,Home Win,1,...,Home Win,1,2,Away Win,2,2,Tie,2,2,Tie
305,34,18/05/2024,13:30,SIGNAL IDUNA PARK,BVB,D98,4,0,Home Win,2,...,Home Win,5,1,Home Win,4,0,Home Win,3,0,Home Win
